<a href="https://colab.research.google.com/github/sckgn/FinalYearProject_ML/blob/main/AdaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AdaBoost is an ensemble algorithm that combines multiple weak learners (algorithms that are generally simple) sequentially. The focus is on improving previously made errors.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset_2018.csv to dataset_2018.csv


In [ ]:
import os 
import pandas as pd
import sklearn
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#Read in ASDTest dataset
asd_2018_df = pd.read_csv("dataset_2018.csv")

asd_2018_df.info()

In [3]:
asd_2018_df.rename(columns={'Class/ASD Traits ':'ASD_Traits'},inplace=True)

In [ ]:
mixed_df = asd_2018_df.drop(['Case_No','Qchat-10-Score','Who completed the test'],axis=1)
mixed_df.columns
mixed_df.head()

In [5]:
features=list(mixed_df)

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

encoder = OneHotEncoder()
scaler = MinMaxScaler()

age = ['Age_Mons']
char_vals = ['ASD_Traits','Sex','Jaundice','Family_mem_with_ASD','Ethnicity']
target = ['ASD_Traits']

In [7]:
def preprocess(age, char_vals,dataframe):
  dataframe[age] = scaler.fit_transform(dataframe[age].values.reshape(-1,1)) 

  for c in char_vals:
      dataframe[c] = encoder.fit_transform(dataframe[c].values.reshape(-1,1)).toarray()
  print(dataframe.dtypes)
  print(dataframe.head())

In [ ]:
preprocess(age,char_vals,mixed_df)

In [9]:
from sklearn.model_selection import train_test_split

X_mixed, y_mixed = train_test_split(mixed_df, test_size=0.3, random_state=42)

X_mixed_train = X_mixed.drop("ASD_Traits", axis=1)
y_mixed_train = X_mixed["ASD_Traits"]

X_mixed_test = y_mixed.drop("ASD_Traits", axis=1)
y_mixed_test = y_mixed["ASD_Traits"]

In [50]:
from numpy import mean, std
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, cross_validate
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()

ada_mixed = ada.fit(X_mixed_train, y_mixed_train)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(ada_mixed, X_mixed_train, y_mixed_train, scoring='accuracy',cv=cv, n_jobs=-1)

print('AdaBoostClassifier Mixed Dataset accuracy:  %.3f (%.3f)' % (mean(scores),std(scores)))

AdaBoostClassifier Mixed Dataset accuracy:  1.000 (0.000)


In [27]:
importance = ada_mixed.feature_importances_

for i,v in enumerate(importance):
  print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.10000
Feature: 1, Score: 0.10000
Feature: 2, Score: 0.08000
Feature: 3, Score: 0.10000
Feature: 4, Score: 0.08000
Feature: 5, Score: 0.10000
Feature: 6, Score: 0.10000
Feature: 7, Score: 0.12000
Feature: 8, Score: 0.10000
Feature: 9, Score: 0.10000
Feature: 10, Score: 0.00000
Feature: 11, Score: 0.00000
Feature: 12, Score: 0.00000
Feature: 13, Score: 0.02000
Feature: 14, Score: 0.00000


In [12]:
from sklearn.metrics import accuracy_score

y_mixed_pred = ada_mixed.predict(X_mixed_test)
print("Accuracy:", accuracy_score(y_mixed_test, y_mixed_pred))

Accuracy: 1.0


**Train, test, validation split**

In [ ]:
new_mixed_df = asd_2018_df.drop(['Case_No','Qchat-10-Score','Who completed the test'],axis=1)
new_mixed_df.columns
new_mixed_df.head()

In [ ]:
preprocess(age, char_vals, new_mixed_df)

In [42]:
X_new = new_mixed_df.drop(['ASD_Traits'], axis=1)
y_new = new_mixed_df['ASD_Traits']

X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_new, y_new, test_size=0.30, random_state=42, stratify=y_new)

In [44]:
X_new_validate, X_new_test, y_new_validate, y_new_test = train_test_split(X_new_test, y_new_test, test_size=0.50, random_state=42, stratify=y_new_test)

In [49]:
new_ada = AdaBoostClassifier()

new_ada.fit(X_new_train, y_new_train)

AdaBoostClassifier()

In [55]:
def cross_validation_results(model, predictors, target, metrics,cv):
  results = cross_validate(model, predictors, target, scoring = metrics, return_train_score=False, cv=10)

  print("Mean Test R2: ", results['test_r2'].mean())
  rmse = np.sqrt(-results['test_neg_mean_squared_error']).mean()
  print("Mean Test MSE: ", -results['test_neg_mean_squared_error'].mean())
  print("Test std: ",np.sqrt(-results['test_neg_mean_squared_error']).std() )
  print("RMSE Test: ", rmse)
  print("Accuracy: ", results['test_accuracy'].mean())

In [56]:
metrics =['r2','neg_mean_squared_error','accuracy']

In [57]:
cross_validation_results(new_ada, X_new_train, y_new_train,metrics, cv=10)

Mean Test R2:  1.0
Mean Test MSE:  -0.0
Test std:  0.0
RMSE Test:  0.0
Accuracy:  1.0


In [58]:
new_val_pred = new_ada.predict(X_new_validate)
print('Accuracy:', accuracy_score(y_new_validate, new_val_pred))

Accuracy: 1.0


**1:1 male to female ratio**

In [ ]:
f_only = asd_2018_df[asd_2018_df['Sex']=='f']
f_only.info()
f_only['ASD_Traits'].value_counts()

In [ ]:
y_male = asd_2018_df[(asd_2018_df['Sex']=='m') & (asd_2018_df['ASD_Traits']=='Yes')].head(194)
y_male.info()
y_male['ASD_Traits'].value_counts()

In [ ]:
n_male = asd_2018_df[(asd_2018_df['Sex']=='m') & (asd_2018_df['ASD_Traits']=='No')].head(125)
n_male.info()
n_male['ASD_Traits'].value_counts()

In [ ]:
equal_male = pd.concat([y_male,n_male], axis=0)
equal_male.info()

In [ ]:
equal_ratio = pd.concat([f_only, equal_male], axis=0)
equal_ratio.info()

In [18]:
equal_ratio = equal_ratio.drop(['Case_No','Qchat-10-Score','Who completed the test'], axis=1)

In [ ]:
preprocess(age, char_vals,equal_ratio)

In [20]:
X_equal, y_equal = train_test_split(equal_ratio, test_size=0.3, random_state=42)

X_equal_train = X_equal.drop("ASD_Traits", axis=1)
y_equal_train = X_equal["ASD_Traits"]

X_equal_test = y_equal.drop("ASD_Traits", axis=1)
y_equal_test = y_equal["ASD_Traits"]

In [21]:
ada_equal = AdaBoostClassifier()
ada_equal = ada_equal.fit(X_equal_train, y_equal_train)

scores_equal = cross_val_score(ada_equal, X_equal_train, y_equal_train, scoring='accuracy',cv=cv, n_jobs=-1,error_score='raise')

print('AdaBoostClassifier Equal Dataset accuracy:  %.3f (%.3f)' % (mean(scores_equal),std(scores_equal)))

AdaBoostClassifier Equal Dataset accuracy:  0.996 (0.011)


In [22]:
y_equal_pred = ada_equal.predict(X_equal_test)
print("Accuracy: ", accuracy_score(y_equal_test, y_equal_pred))

Accuracy:  1.0


**Female Only**

In [23]:
f_only.head()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,ASD_Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes
12,13,0,0,0,0,0,0,0,0,0,0,25,0,f,middle eastern,yes,no,family member,No
13,14,1,1,1,1,0,0,1,0,1,1,15,7,f,middle eastern,yes,no,family member,Yes
17,18,1,1,1,0,1,1,1,1,0,1,12,8,f,middle eastern,yes,no,family member,Yes


In [28]:
f_only = f_only.drop(["Case_No","Qchat-10-Score","Who completed the test"], axis=1)

In [ ]:
preprocess(age, char_vals,f_only)

In [29]:
X_f, y_f = train_test_split(f_only, test_size=0.3, random_state=42)

X_f_train = X_f.drop("ASD_Traits", axis=1)
y_f_train = X_f["ASD_Traits"]

X_f_test = y_f.drop("ASD_Traits", axis=1)
y_f_test = y_f["ASD_Traits"]

In [30]:
ada_f = AdaBoostClassifier()
ada_f_model = ada_f.fit(X_f_train, y_f_train)
scores_f = cross_val_score(ada_f_model, X_f_train, y_f_train, scoring='accuracy',cv=cv, n_jobs=-1,error_score='raise')
print('AdaBoostClassifier Female Dataset accuracy:  %.3f (%.3f)' % (mean(scores_f),std(scores_f)))

AdaBoostClassifier Female Dataset accuracy:  0.970 (0.037)


In [31]:
y_f_pred = ada_mixed.predict(X_f_test)
print("Accuracy:", accuracy_score(y_f_test, y_f_pred))

Accuracy: 1.0


In [32]:
y_new_f_pred = ada_f_model.predict(X_f_test)
print("Accuracy:", accuracy_score(y_f_test,y_new_f_pred))

Accuracy: 0.9166666666666666
